# IMPORT

In [10]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import GPyOpt
from timeit import default_timer as timer

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

# NEW DATA

In [14]:
# define number of classes
train_pr_label = 50

# load train and test sets
train = pd.read_csv('Data/fashion-mnist_train.csv').to_numpy()
test = pd.read_csv('Data/fashion-mnist_test.csv').to_numpy()

X_train = train[:,1:]
y_train = train[:,0]
X_test = test[:,1:]
y_test = test[:,0]

where_train = []
where_test = []
where_pool = []

for label in range(10):
    where_train.append(np.where(y_train == label)[0][:train_pr_label])
    where_test.append(np.where(y_test == label)[0])
    where_pool.append(np.where(y_train == label)[0][train_pr_label:])

def flatten(array):
    new_array = []
    for sublist in array:
        for item in sublist:
            new_array.append(item)
    return new_array
    
where_train = flatten(where_train)
where_test = flatten(where_test)
where_pool = flatten(where_pool)

X_pool = X_train[where_pool]
y_pool = y_train[where_pool]

X_train = X_train[where_train]
y_train = y_train[where_train]

X_test = X_test[where_test]
y_test = y_test[where_test]
    
print("Train data shape:", X_train.shape, "Test data shape:", X_test.shape)
print("Train labels shape:", y_train.shape,"  Test labels shape:", y_test.shape)
print("Pool data shape:", X_pool.shape,"  Pool labels shape:", y_pool.shape)

Train data shape: (500, 784) Test data shape: (10000, 784)
Train labels shape: (500,)   Test labels shape: (10000,)
Pool data shape: (59500, 784)   Pool labels shape: (59500,)


# NAIVE BAYES TEST

In [15]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (y_test != y_pred).sum()))
print("{0}% mislabeled data.".format(((y_test != y_pred).sum()/X_test.shape[0])*100))

mislabeled = [0,0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0,0]
for i in range(len(y_pred)):
    count[int(y_test[i])] += 1
    if y_pred[i] != y_test[i]:
        mislabeled[int(y_test[i])] += 1

for i in range(10):
    print("Mislabeled of class {0}: {1}".format(i,(mislabeled[i]/count[i])*100))

Number of mislabeled points out of a total 10000 points : 2549
25.490000000000002% mislabeled data.
Mislabeled of class 0: 27.0
Mislabeled of class 1: 4.0
Mislabeled of class 2: 38.4
Mislabeled of class 3: 28.599999999999998
Mislabeled of class 4: 26.900000000000002
Mislabeled of class 5: 36.3
Mislabeled of class 6: 55.300000000000004
Mislabeled of class 7: 16.2
Mislabeled of class 8: 14.000000000000002
Mislabeled of class 9: 8.200000000000001
